In [20]:
import numpy as np
import pandas as pd
import scipy
from util import (BOHR, read_mat, read_comp, get_iso, get_aniso, read_by_prompt, get_df_err, get_rmsre_3comp, get_relrmsd_3comp)
import itertools
import warnings
import basis_set_exchange as bse
from functools import partial
import copy

warnings.filterwarnings("ignore")
np.set_printoptions(8, suppress=True, linewidth=150)
pd.set_option('display.max_rows', None)
pd.set_option("display.precision", 3)
pd.set_option("float_format", '{:.3f}'.format)

In [21]:
from pyscf import dft

In [6]:
import matplotlib.pyplot as plt
from matplotlib_inline.backend_inline import set_matplotlib_formats
%matplotlib inline

set_matplotlib_formats('svg')

In [7]:
def get_df_iso(df):
    xx, yy, zz = df["xx"], df["yy"], df["zz"]
    return 1 / 3 * (xx + yy + zz)

def get_df_aniso(df):
    xx, yy, zz, xy, yz, zx = df["xx"], df["yy"], df["zz"], df["xy"], df["yz"], df["zx"]
    return np.sqrt(0.5) * ((xx - yy)**2 + (yy - zz)**2 + (zz - xx)**2 + 6 * (xy**2 + yz**2 + zx**2))**0.5

## 读取基本数据

In [8]:
df_bench_iso = pd.read_csv("result-iso.csv", index_col=[0, 1], header=[0, 1])
df_bench_aniso = pd.read_csv("result-aniso.csv", index_col=[0, 1], header=[0, 1])
df_bench_comp = pd.read_csv("result-comp.csv", index_col=[0, 1], header=[0, 1, 2])

In [9]:
df_xdh_raw = pd.read_csv("raw_data/xdh-b3lyp_components.csv", index_col=[0, 1], header=[0, 1])
df_xdh_raw.index = pd.MultiIndex.from_tuples([((l[0], l[1].replace("_", "-")) if l[0] == "HR46" else (l[0], f"{int(l[1]):04d}") if l[0] == "T145" else l) for l in df_xdh_raw.index])

In [10]:
df_xdh = pd.DataFrame(index=df_bench_iso.index, columns=df_xdh_raw.columns)

for dataset, dataset_origin in [("HH101 (NSP)", "HH118"), ("HH101 (SP)", "HH118"), ("HR46", "HR46"), ("T144", "T145")]:
    for mol in df_xdh.loc[dataset].index:
        df_xdh.loc[(dataset, mol)] = df_xdh_raw.loc[(dataset_origin, mol)]

In [11]:
components = [l[0] for l in df_xdh_raw.columns if l[1] == "xx"]
# I forgot to exchange FH-OH and H2O-Li xx<->zz
for comp in components:
    for idx in [("HH101 (SP)", "FH-OH"), ("HH101 (SP)", "H2O-Li")]:
        xx, zz = df_xdh.loc[idx, [(comp, "xx"), (comp, "zz")]]
        df_xdh.loc[idx, (comp, "zz")] = xx
        df_xdh.loc[idx, (comp, "xx")] = zz

In [12]:
### IMPORTANT ###
# data of following should be changed when more DH functionals included.

df_res_wt = pd.Series(
    index=pd.MultiIndex.from_tuples(
          [("isotropic", dat) for dat in ("HR46", "T144", "HH101 (NSP)", "HH101 (SP)")]
        + [("anisotropic", dat) for dat in ("HR46", "T144")]
        + [("components", dat) for dat in ("HH101 (NSP)", "HH101 (SP)")]),
    data=[
        0.061837091690685, 0.054878115055414, 0.038215111754263, 0.032692374803261,
        0.024584977413508, 0.025091475490632, 0.037099844927297, 0.029890098744748]
)

## 处理基本数据

In [13]:
para_xyg3 = {
    "noxc": 1,
    "HF": 0.8033,
    "LDA_X": -0.0140,
    "GGA_X_B88": 0.2107,
    "LDA_C_VWN_RPA": 0,
    "GGA_C_LYP": 0.6789,
    "MP2_OS": 0.3211,
    "MP2_SS": 0.3211,
}

In [14]:
def get_pol_by_para(df, para):
    return sum([val * df[key] for (key, val) in para.items()])

def get_wtmad_by_para(para):
    df_err = pd.Series(index=pd.MultiIndex.from_tuples(list(df_res_wt.index) + [("wtmad", "wtmad")]))
    df_pol = get_pol_by_para(df_xdh, para)
    
    for dat in df_err["isotropic"].index:
        df_err[("isotropic", dat)] = get_df_err(get_df_iso(df_pol.loc[dat]), df_bench_iso.loc[dat, ("WFT", "CCSD(T)")])["RelRMSD/%"]
    for dat in df_err["anisotropic"].index:
        df_err[("anisotropic", dat)] = get_df_err(get_df_aniso(df_pol.loc[dat]), df_bench_aniso.loc[dat, ("WFT", "CCSD(T)")])["RelRMSD/%"]
    for dat in df_err["components"].index:
        df_err[("components", dat)] = get_relrmsd_3comp(get_df_err(
            df_pol.loc[dat][["xx", "yy", "zz"]], df_bench_comp.loc[dat, ("WFT", "CCSD(T)")][["xx", "yy", "zz"]]))
    df_err.loc[("wtmad", "wtmad")] = (df_err * df_res_wt).sum()
    return df_err

In [15]:
get_wtmad_by_para(para_xyg3)

isotropic    HR46          1.083
             T144          1.385
             HH101 (NSP)   1.345
             HH101 (SP)    2.047
anisotropic  HR46          2.513
             T144          3.141
components   HH101 (NSP)   1.456
             HH101 (SP)    2.272
wtmad        wtmad         0.524
dtype: float64

## 优化模块

In [68]:
def para_opt(func, v0):
    v = np.array(v0).copy()
    v_min = None
    fun_min = 100000
    res = None
    for seed in range(20):
        np.random.seed(seed)
        if res is not None:
            v = res.x + (np.random.random(len(v)) - 0.5)
        res = scipy.optimize.minimize(func, v, method="L-BFGS-B", options={"gtol": 1e-5})
        print(("{:10.6f}" * len(res.x)).format(*res.x), "|", "{:10.6f}".format(res.fun), "|", res.success)
        if res.fun < fun_min:
            fun_min = res.fun
            v_min = res.x
    print(("{:10.6f}" * len(res.x)).format(*res.x), "|", "{:10.6f}".format(res.fun), "|", "Final")
    func(v_min, verbose=True)

## 优化

### B3LYP

In [69]:
def get_b3lyp_rpa_wtmad_by_arr(v, verbose=False):
    # [0.2, 0.08, 0.19]
    para = {
        "noxc": 1,
        "HF": v[0],
        "LDA_X": v[1],
        "GGA_X_B88": 1 - v[0] - v[1],
        "LDA_C_VWN_RPA": v[2],
        "GGA_C_LYP": 1 - v[2],
    }
    df_err = get_wtmad_by_para(para)
    if verbose:
        print(("{:12.6f}" * 3).format(*np.array(v)), "|", ("{:12.6f}" * 9).format(*np.array(df_err)))
    return df_err[-1]

In [70]:
para_opt(get_b3lyp_rpa_wtmad_by_arr, [0.2, 0.08, 0.19])

  0.330646  0.113675  0.741727 |   0.883752 | True
  0.330646  0.113679  0.741733 |   0.883752 | True
  0.330647  0.113676  0.741724 |   0.883752 | True
  0.330647  0.113678  0.741725 |   0.883752 | True
  0.330647  0.113686  0.741730 |   0.883752 | True
  0.330647  0.113678  0.741727 |   0.883752 | True
  0.330647  0.113677  0.741723 |   0.883752 | True
  0.330651  0.113672  0.741695 |   0.883752 | True
  0.330647  0.113678  0.741724 |   0.883752 | True
  0.330655  0.113634  0.741644 |   0.883752 | True
  0.330647  0.113676  0.741725 |   0.883752 | True
  0.330647  0.113681  0.741727 |   0.883752 | True
  0.330647  0.113676  0.741725 |   0.883752 | True
  0.330649  0.113677  0.741715 |   0.883752 | True
  0.330649  0.113686  0.741713 |   0.883752 | True
  0.330647  0.113677  0.741723 |   0.883752 | True
  0.330647  0.113671  0.741718 |   0.883752 | True
  0.330647  0.113677  0.741724 |   0.883752 | True
  0.330647  0.113676  0.741726 |   0.883752 | True
  0.330647  0.113677  0.741724 

In [ ]:
1.365399    2.101874    2.306828    3.576636    5.201311    6.006971    2.397889    3.724622    0.883752

### XYG3

In [72]:
def get_xyg3_wtmad_by_arr(v, verbose=False):
    # [0.8033, -0.0140, 0.3211]
    para = {
        "noxc": 1,
        "HF": v[0],
        "LDA_X": 1 - v[0] - v[1],
        "GGA_X_B88": v[1],
        "LDA_C_VWN_RPA": 0,
        "GGA_C_LYP": 1 - v[2],
        "MP2_OS": v[2],
        "MP2_SS": v[2],
    }
    df_err = get_wtmad_by_para(para)
    if verbose:
        print(("{:12.6f}" * 3).format(*np.array(v)), "|", ("{:12.6f}" * 9).format(*np.array(df_err)))
    return df_err[-1]

In [73]:
v0 = [0.8033, 0.2107, 0.3211]
para_opt(get_xyg3_wtmad_by_arr, v0)

  0.887385 -0.185307  0.340747 |   0.471308 | True
  0.887385 -0.185309  0.340748 |   0.471308 | True
  0.887385 -0.185309  0.340748 |   0.471308 | True
  0.887385 -0.185309  0.340748 |   0.471308 | True
  0.887385 -0.185309  0.340748 |   0.471308 | True
  0.887388 -0.185333  0.340748 |   0.471308 | True
  0.887385 -0.185308  0.340748 |   0.471308 | True
  0.887385 -0.185310  0.340748 |   0.471308 | True
  0.887385 -0.185309  0.340748 |   0.471308 | True
  0.887385 -0.185309  0.340748 |   0.471308 | True
  0.887384 -0.185306  0.340748 |   0.471308 | True
  0.887384 -0.185306  0.340748 |   0.471308 | True
  0.887385 -0.185310  0.340748 |   0.471308 | True
  0.887385 -0.185309  0.340748 |   0.471308 | True
  0.887381 -0.185322  0.340744 |   0.471308 | True
  0.887387 -0.185308  0.340750 |   0.471308 | True
  0.887391 -0.185332  0.340751 |   0.471308 | True
  0.887383 -0.185305  0.340747 |   0.471308 | True
  0.887382 -0.185302  0.340746 |   0.471308 | True
  0.887386 -0.185311  0.340749 

### XYG5

In [78]:
def get_xyg5_wtmad_by_arr(v, verbose=False):
    # [0.9150, 0.0238, 0.4957, 0.4548, 0.2764]
    para = {
        "noxc": 1,
        "HF": v[0],
        "LDA_X": 1 - v[0] - v[1],
        "GGA_X_B88": v[1],
        "LDA_C_VWN_RPA": 0,
        "GGA_C_LYP": v[2],
        "MP2_OS": v[3],
        "MP2_SS": v[4],
    }
    df_err = get_wtmad_by_para(para)
    if verbose:
        print(("{:12.6f}" * 3).format(*np.array(v)), "|", ("{:12.6f}" * 9).format(*np.array(df_err)))
    return df_err[-1]

In [75]:
v0 = [0.9150, 0.0238, 0.4957, 0.4548, 0.2764]
para_opt(get_xyg5_wtmad_by_arr, v0)

  0.843480 -0.075137  0.615211  0.408184  0.115199 |   0.388153 | True
  0.843482 -0.075140  0.615207  0.408185  0.115201 |   0.388153 | True
  0.843477 -0.075129  0.615212  0.408183  0.115196 |   0.388153 | True
  0.843480 -0.075136  0.615210  0.408184  0.115199 |   0.388153 | True
  0.843481 -0.075142  0.615208  0.408185  0.115199 |   0.388153 | True
  0.843479 -0.075137  0.615212  0.408184  0.115198 |   0.388153 | True
  0.843475 -0.075116  0.615220  0.408182  0.115198 |   0.388153 | True
  0.843480 -0.075136  0.615211  0.408184  0.115199 |   0.388153 | True
  0.843479 -0.075135  0.615212  0.408184  0.115198 |   0.388153 | True
  0.843479 -0.075137  0.615209  0.408179  0.115205 |   0.388153 | True
  0.843479 -0.075136  0.615212  0.408184  0.115199 |   0.388153 | True
  0.843480 -0.075136  0.615210  0.408184  0.115199 |   0.388153 | True
  0.843480 -0.075136  0.615211  0.408184  0.115199 |   0.388153 | True
  0.843479 -0.075132  0.615212  0.408184  0.115199 |   0.388153 | True
  0.84

### XYG6

In [79]:
def get_xyg6_wtmad_by_arr(v, verbose=False):
    # [0.9105, -0.0681, 0.1800, 0.2244, 0.4695, 0.2589]
    para = {
        "noxc": 1,
        "HF": v[0],
        "LDA_X": 1 - v[0] - v[1],
        "GGA_X_B88": v[1],
        "LDA_C_VWN_RPA": v[2],
        "GGA_C_LYP": v[3],
        "MP2_OS": v[4],
        "MP2_SS": v[5],
    }
    df_err = get_wtmad_by_para(para)
    if verbose:
        print(("{:12.6f}" * 3).format(*np.array(v)), "|", ("{:12.6f}" * 9).format(*np.array(df_err)))
    return df_err[-1]

In [80]:
v0 = [0.9105, -0.0681, 0.1800, 0.2244, 0.4695, 0.2589]
para_opt(get_xyg6_wtmad_by_arr, v0)

  0.707770 -0.228740  0.995121 -0.667078  0.365673  0.049561 |   0.346923 | True
  0.707760 -0.228758  0.995192 -0.667153  0.365677  0.049544 |   0.346923 | True
  0.707756 -0.228754  0.995208 -0.667175  0.365675  0.049540 |   0.346923 | True
  0.707762 -0.228801  0.995218 -0.667175  0.365674  0.049554 |   0.346923 | True
  0.707761 -0.228760  0.995180 -0.667135  0.365678  0.049543 |   0.346923 | True
  0.707758 -0.228739  0.995173 -0.667110  0.365677  0.049546 |   0.346923 | True
  0.707755 -0.228753  0.995201 -0.667160  0.365674  0.049541 |   0.346923 | True
  0.707770 -0.228766  0.995144 -0.667098  0.365677  0.049557 |   0.346923 | True
  0.707744 -0.228768  0.995275 -0.667258  0.365670  0.049536 |   0.346923 | True
  0.707733 -0.228699  0.995230 -0.667179  0.365657  0.049541 |   0.346923 | True
  0.707756 -0.228760  0.995217 -0.667188  0.365676  0.049541 |   0.346923 | True
  0.707736 -0.228719  0.995282 -0.667232  0.365665  0.049546 |   0.346923 | True
  0.707759 -0.228757  0.9951

In [ ]:
0.560103    0.793545    0.878909    1.287687    2.290330    2.252288    1.020381    1.417789    0.346923

### XYG7

In [81]:
def get_xyg7_wtmad_by_arr(v, verbose=False):
    # [0.8971, 0.2055, -0.1408, 0.4056, 0.1159, 0.4052, 0.2589]
    para = {
        "noxc": 1,
        "HF": v[0],
        "LDA_X": v[1],
        "GGA_X_B88": v[2],
        "LDA_C_VWN_RPA": v[3],
        "GGA_C_LYP": v[4],
        "MP2_OS": v[5],
        "MP2_SS": v[6],
    }
    df_err = get_wtmad_by_para(para)
    if verbose:
        print(("{:12.6f}" * 3).format(*np.array(v)), "|", ("{:12.6f}" * 9).format(*np.array(df_err)))
    return df_err[-1]

In [83]:
v0 = [0.8971, 0.2055, -0.1408, 0.4056, 0.1159, 0.4052, 0.2589]
para_opt(get_xyg7_wtmad_by_arr, v0)

  0.713012  0.528604 -0.263555  1.293849 -0.834458  0.370935  0.036979 |   0.341906 | True
  0.712984  0.528543 -0.263460  1.293781 -0.834465  0.370905  0.036983 |   0.341906 | True
  0.713016  0.528605 -0.263555  1.293766 -0.834413  0.370938  0.036981 |   0.341906 | True
  0.713003  0.528597 -0.263539  1.293862 -0.834475  0.370930  0.036976 |   0.341906 | True
  0.713115  0.528718 -0.263749  1.293044 -0.833534  0.371030  0.036947 |   0.341906 | True
  0.713013  0.528606 -0.263560  1.293855 -0.834453  0.370937  0.036976 |   0.341906 | True
  0.713000  0.528626 -0.263567  1.293919 -0.834537  0.370933  0.036967 |   0.341906 | True
  0.713351  0.529170 -0.264456  1.293299 -0.833904  0.371142  0.037120 |   0.341906 | True
  0.713012  0.528601 -0.263553  1.293848 -0.834452  0.370935  0.036977 |   0.341906 | True
  0.711153  0.533184 -0.266855  1.315623 -0.856440  0.370395  0.035686 |   0.341919 | True
  0.713013  0.528604 -0.263556  1.293840 -0.834448  0.370935  0.036979 |   0.341906 | True

### XYGJ-OS

In [89]:
def get_xygjos_wtmad_by_arr(v, verbose=False):
    # [0.7731, 0.2309, 0.2754, 0.4364]
    para = {
        "noxc": 1,
        "HF": v[0],
        "LDA_X": 1 - v[0],
        "GGA_X_B88": 0,
        "LDA_C_VWN_RPA": v[1],
        "GGA_C_LYP": v[2],
        "MP2_OS": v[3],
    }
    df_err = get_wtmad_by_para(para)
    if verbose:
        print(("{:12.6f}" * 3).format(*np.array(v)), "|", ("{:12.6f}" * 9).format(*np.array(df_err)))
    return df_err[-1]

In [88]:
v0 = [0.7731, 0.2309, 0.2754, 0.4364]
para_opt(get_xygjos_wtmad_by_arr, v0)

  0.667163  0.883241 -0.500749  0.353082 |   0.360344 | True
  0.667164  0.883239 -0.500747  0.353082 |   0.360344 | True
  0.667167  0.883237 -0.500749  0.353085 |   0.360344 | True
  0.667163  0.883241 -0.500749  0.353082 |   0.360344 | True
  0.667163  0.883241 -0.500748  0.353082 |   0.360344 | True
  0.667164  0.883239 -0.500747  0.353082 |   0.360344 | True
  0.667164  0.883239 -0.500747  0.353082 |   0.360344 | True
  0.667163  0.883241 -0.500747  0.353081 |   0.360344 | True
  0.667163  0.883244 -0.500752  0.353081 |   0.360344 | True
  0.667165  0.883236 -0.500746  0.353083 |   0.360344 | True
  0.667164  0.883239 -0.500747  0.353082 |   0.360344 | True
  0.667165  0.883236 -0.500747  0.353084 |   0.360344 | True
  0.667164  0.883241 -0.500749  0.353082 |   0.360344 | True
  0.667165  0.883234 -0.500744  0.353083 |   0.360344 | True
  0.667164  0.883241 -0.500749  0.353083 |   0.360344 | True
  0.667167  0.883224 -0.500730  0.353084 |   0.360344 | True
  0.667164  0.883245 -0.

### XYG-OS5

In [90]:
def get_xygos5_wtmad_by_arr(v, verbose=False):
    # [0.8928, -0.2321, 0.3268, -0.0635, 0.5574]
    para = {
        "noxc": 1,
        "HF": v[0],
        "LDA_X": 1 - v[0],
        "GGA_X_B88": v[1],
        "LDA_C_VWN_RPA": v[2],
        "GGA_C_LYP": v[3],
        "MP2_OS": v[4],
    }
    df_err = get_wtmad_by_para(para)
    if verbose:
        print(("{:12.6f}" * 3).format(*np.array(v)), "|", ("{:12.6f}" * 9).format(*np.array(df_err)))
    return df_err[-1]

In [91]:
v0 = [0.8928, -0.2321, 0.3268, -0.0635, 0.5574]
para_opt(get_xygos5_wtmad_by_arr, v0)

  0.681865 -0.020577  1.120201 -0.614128  0.359351 |   0.355572 | True
  0.681864 -0.020579  1.120238 -0.614159  0.359350 |   0.355572 | True
  0.681864 -0.020578  1.120224 -0.614147  0.359351 |   0.355572 | True
  0.681864 -0.020578  1.120226 -0.614148  0.359351 |   0.355572 | True
  0.681866 -0.020574  1.120117 -0.614046  0.359347 |   0.355572 | True
  0.681864 -0.020578  1.120225 -0.614148  0.359351 |   0.355572 | True
  0.681864 -0.020579  1.120227 -0.614149  0.359351 |   0.355572 | True
  0.681864 -0.020579  1.120234 -0.614161  0.359351 |   0.355572 | True
  0.681864 -0.020578  1.120222 -0.614146  0.359350 |   0.355572 | True
  0.681864 -0.020579  1.120229 -0.614152  0.359351 |   0.355572 | True
  0.681869 -0.020577  1.120184 -0.614113  0.359352 |   0.355572 | True
  0.681864 -0.020578  1.120225 -0.614148  0.359351 |   0.355572 | True
  0.681864 -0.020578  1.120213 -0.614139  0.359351 |   0.355572 | True
  0.681863 -0.020577  1.120214 -0.614141  0.359350 |   0.355572 | True
  0.68

### XYG-OS3

In [92]:
def get_xygos3_wtmad_by_arr(v, verbose=False):
    # [0.7731, 0.2754, 0.4364]
    para = {
        "noxc": 1,
        "HF": v[0],
        "LDA_X": 1 - v[0],
        "GGA_X_B88": 0,
        "LDA_C_VWN_RPA": 0,
        "GGA_C_LYP": v[1],
        "MP2_OS": v[2],
    }
    df_err = get_wtmad_by_para(para)
    if verbose:
        print(("{:12.6f}" * 3).format(*np.array(v)), "|", ("{:12.6f}" * 9).format(*np.array(df_err)))
    return df_err[-1]

In [93]:
v0 = [0.7731, 0.2754, 0.4364]
para_opt(get_xygos3_wtmad_by_arr, v0)

  0.794150  0.673447  0.419592 |   0.405278 | True
  0.794151  0.673447  0.419592 |   0.405278 | True
  0.794150  0.673448  0.419592 |   0.405278 | True
  0.794151  0.673447  0.419592 |   0.405278 | True
  0.794150  0.673449  0.419592 |   0.405278 | True
  0.794151  0.673447  0.419592 |   0.405278 | True
  0.794151  0.673447  0.419592 |   0.405278 | True
  0.794151  0.673437  0.419591 |   0.405278 | True
  0.794151  0.673448  0.419592 |   0.405278 | True
  0.794151  0.673447  0.419592 |   0.405278 | True
  0.794151  0.673447  0.419592 |   0.405278 | True
  0.794151  0.673448  0.419592 |   0.405278 | True
  0.794151  0.673445  0.419592 |   0.405278 | True
  0.794151  0.673448  0.419592 |   0.405278 | True
  0.794152  0.673444  0.419593 |   0.405278 | True
  0.794150  0.673447  0.419592 |   0.405278 | True
  0.794151  0.673446  0.419592 |   0.405278 | True
  0.794151  0.673447  0.419592 |   0.405278 | True
  0.794153  0.673443  0.419593 |   0.405278 | True
  0.794151  0.673447  0.419592 

In [ ]:
0.719953    0.667872    0.913041    2.117647    2.565388    1.853147    1.118469    2.305794    0.405278